# R5.A.08 Qualite de developpement

## Projet Test - Selenium Python

Breville Alexandre Groupe Din - BUT3 Informatique

Le projet présenté ici est destiné à tester le site internet : https://ztrain-web.vercel.app/ à l'aide de Selenium, j'utiliserai la version Python pour ce projet, je ferais en sorte d'expliquer chaque étape dans les cellules de Markdown afin de faciliter la compréhension du code et mes explications.

## Étape 1 - Installation de Selenium

In [1]:
!pip install --upgrade pip
!pip install selenium
print("ok")

## Étape 2 - Importation des services

Afin de pouvoir faire nos tests, nous avons besoin de quelque chose qui puisse émuler une page Internet afin que toutes nos fonctions puissent être executer sans problèmes et pour cela nous avons besoin d'un webdriver, j'utiliserai **Firefox** en tant que webdriver pour ce projet.

In [2]:
import time
import json

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

## Étape 3 - Mettre le driver dans une variable et ouvrir la fenêtre

Ce qu'on va faire c'est mettre notre **webdriver** dans une variable *browser* afin de pouvoir l'utiliser plus facilement ensuite grâce à la méthode `get()`, nous allons ouvrir la page.

In [4]:
browser = webdriver.Firefox()
browser.get("https://ztrain-web.vercel.app/")
time.sleep(1)
browser.maximize_window()

Après avoir réussi à installer et importer selenium on peut désormais procéder à quelques tests, on va d'abord faire des tests simples pour ensuite se dirigier sur des tests plus complexes.

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

## Étape 4 - Rechercher des éléments

Nos 2 premiers tests vont être de vérifier si lorsque l'on cherche des éléments ils apparaissent ou non. Nous avons 2 tableaux `True_tab` et `False_tab`, ils contiennent des éléments qui sont dans le magasins et d'autres qui ne le sont pas. 

In [6]:
search = browser.find_element(By.ID,"style_input_navbar_search__Scaxy")

True_tab = ["T-shirt","Chaussures","PC Portable"]
False_tab = ["Manteau","Test","123456"]

for i in range(len(True_tab)):
    search.clear()
    search.send_keys(True_tab[i])

    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="style_popular_product_wrapper__z6J0h"]')))

    results = browser.find_elements(By.XPATH, '//*[@id="style_popular_product_wrapper__z6J0h"]')

    try:
        assert any(True_tab[i].lower() in result.text.lower() for result in results)
        print(f"La recherche pour {True_tab[i]} a réussi.")
    except AssertionError as e:
        print(f"Échec de la recherche pour {True_tab[i]}: {e}")
    time.sleep(2)

for i in range(len(False_tab)):
    search.clear()
    search.send_keys(False_tab[i])

    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="style_popular_product_wrapper__z6J0h"]')))

    empty_result = browser.find_element(By.XPATH, '//*[@id="style_empty_result___y6P_"]')

    try:
        assert empty_result.is_displayed()
        print(f"{False_tab[i]} n'est pas dans le recherche proposé")
    except AssertionError as e:
        print(f"Échec de la recherche pour {False_tab[i]}: {e}")
    time.sleep(2)

La recherche pour T-shirt a réussi.
La recherche pour Chaussures a réussi.
La recherche pour PC Portable a réussi.
Manteau n'est pas dans le recherche proposé
Test n'est pas dans le recherche proposé
123456 n'est pas dans le recherche proposé


## Étape 5 - S'inscrire

Comme autre test, je vais essayer de m'inscrire en ayant au préalable renseigner les informations dans un JSON

In [3]:
connecting = browser.find_element(By.ID,'style_avatar_wrapper__pEGIQ')
connecting.click()

inscription_button = browser.find_element(By.XPATH,'//*[@id="mui-p-4847-T-2"]')
inscription_button.click()

email = browser.find_element(By.XPATH,'//*[@id="email_register"]')
password = browser.find_element(By.XPATH,'//*[@id="password_register"]')
confirm_password = browser.find_element(By.XPATH,'//*[@id="confirm_password_register"]')
validation_button = browser.find_element(By.XPATH,'//*[@id="btn_register"]')

with open("users.json", "r") as json_file:
    loaded_users = json.load(json_file)

for user in loaded_users:
    email.clear()
    password.clear()
    confirm_password.clear()

    email_input = user["email"]
    password_input = user["password"]

    email.send_keys(email_input)
    password.send_keys(password_input)
    confirm_password.send_keys(password_input)

    try:
        validation_button.click()
        error_message = browser.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div/div[2]/div[1]/div/main/div[2]/div[4]/div/div/form/p')
        assert error_message.is_displayed()
        print(f"Erreur détectée pour l'utilisateur avec l'email {email_input} et le mot de passe {password_input}")
    except Exception :
        print(f"Aucune erreur détectée pour l'utilisateur avec l'email {email_input} et le mot de passe {password_input}\n le compte s'est connecté")

    time.sleep(5)

## Étape 6 - Deconnexion 

Ce test permet de voir si un compte peut bien se déconnecter et que le bon logo apparaît.

In [14]:
account = browser.find_element(By.XPATH,'/html/body/div[1]/div/nav/div[4]/p')
account.click()

WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[3]/ul')))

logout = browser.find_element(By.XPATH,'//*[@id="logout"]')
logout.click()

assert connecting.is_displayed()

## Étape 7 - Connexion 

À l'aide du même JSON que l'on a utilisé pour l'inscription, nous allons essayé de nous connecter et refaire les même tests que ceux effectués précedemment

In [10]:
connecting = browser.find_element(By.ID,'style_avatar_wrapper__pEGIQ')
connecting.click()

#connection_button = browser.find_element(By.XPATH,'//*[@id="mui-p-4847-T-1"]')
#connection_button.click()

email = browser.find_element(By.XPATH,'//*[@id="email_login"]')
password = browser.find_element(By.XPATH,'//*[@id="password_login"]')
validation_button = browser.find_element(By.XPATH,'//*[@id="btn_login"]')

with open("users.json", "r") as json_file:
    loaded_users = json.load(json_file)

for user in loaded_users:
    email.clear()
    password.clear()

    email_input = user["email"]
    password_input = user["password"]

    email.send_keys(email_input)
    password.send_keys(password_input)

    try:
        validation_button.click()
        error_message = browser.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div/div[2]/div[1]/div/main/div[2]/div[4]/div/div/form/p')
        assert error_message.is_displayed()
        print(f"Erreur détectée pour l'utilisateur avec l'email {email_input} et le mot de passe {password_input}")
    except Exception as e:
        print(e)
        print(f"Aucune erreur détectée pour l'utilisateur avec l'email {email_input} et le mot de passe {password_input}\n le compte s'est connecté")

    time.sleep(5)

Message: Unable to locate element: /html/body/div[2]/div/div[2]/div/div[2]/div[1]/div/main/div[2]/div[4]/div/div/form/p; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Aucune erreur détectée pour l'utilisateur avec l'email user1 et le mot de passe password1
 le compte s'est connecté
Message: Unable to locate element: /html/body/div[2]/div/div[2]/div/div[2]/div[1]/div/main/div[2]/div[4]/div/div/form/p; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/sha

## Étape 8 - Mettre des éléments dans un panier

Cette fois-ci, on va mettre un élément dans un panier de 2 manières différentes et observer si cela est correct.

In [10]:
lamp = browser.find_element(By.XPATH,'/html/body/div[1]/div/main/div/div/div[1]/div[1]/img')
lamp.click()

add_cart = browser.find_element(By.XPATH,'//*[@id="style_btn_add_cart__gTXM7"]')
add_cart.click()

time.sleep(1)

product_pop_up = browser.find_element(By.XPATH, '/html/body/div[2]/div')
assert product_pop_up.is_displayed()

In [4]:
lamp = browser.find_element(By.XPATH,'/html/body/div[1]/div/main/div/div/div[1]/div[1]/img')
ActionChains(browser).move_to_element(lamp).perform()
lamp_button = browser.find_element(By.XPATH,'/html/body/div[1]/div/main/div/div/div[1]/div[2]/button')

time.sleep(1)
assert lamp_button.is_displayed()
lamp_button.click()
time.sleep(1)

product_pop_up = browser.find_element(By.XPATH, '/html/body/div[2]/div')
assert product_pop_up.is_displayed()

## Étape 9 - Acheter le panier

Quand on achéte un panier, si nous n'avons pas de compte, le site nous invite à nous connecter et dans le cas contraire on peut renseigner ses informations.

Les informations sont dans un JSON.

In [12]:
cart = browser.find_element(By.XPATH,'//*[@id="style_content_cart_wrapper__mqNbf"]')
cart.click()

command_button =  browser.find_element(By.XPATH,'//*[@id="style_btn_cart__zrT9Q"]')
command_button.click()
time.sleep(1)

connection_pop_up = browser.find_element(By.XPATH,'/html/body/div[4]/div/div[2]/div/div[2]')
assert connection_pop_up.is_displayed()

In [24]:
with open("credit_card.json", "r") as json_file:
    data = json.load(json_file)

card = data[0]

cart.click()
command_button.click()
time.sleep(1)

delivery_input = browser.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/main/form/div[1]/div[1]/input')

card_number = browser.find_element(By.XPATH,'//*[@id="card-number"]')
expire_date = browser.find_element(By.XPATH,'//*[@id="card-expiry"]')
cvc = browser.find_element(By.XPATH,'//*[@id="cvc"]')

adress = browser.find_element(By.XPATH,'//*[@id="style_input_address__CrN2C"]')
confirm_button = browser.find_element(By.XPATH,'//*[@id="style_btnSubmit__sn_sg"]')

card_number_input = card["CardNumber"]
expire_date_input = card["Expiry"]
cvc_input = str(card["CVV"])
adress_input = card["Address"]

delivery_input.click()

card_number.clear()
expire_date.clear()
cvc.clear()
adress.clear()

card_number.send_keys(card_number_input)
expire_date.send_keys(expire_date_input)
cvc.send_keys(cvc_input)
adress.send_keys(adress_input)

time.sleep(2)

confirm_button.click()

## Étape 10 Modifier les informations d'un compte

In [9]:
account = browser.find_element(By.XPATH,'/html/body/div[1]/div/nav/div[4]/p')
account.click()

my_account = browser.find_element(By.XPATH,'/html/body/div[2]/div[3]/ul/li[3]/span[2]/a')
my_account.click()
time.sleep(1)

nom = browser.find_element(By.XPATH,'//*[@id="lastName"]')
prenom = browser.find_element(By.XPATH,'//*[@id="firstName"]')
adress = browser.find_element(By.XPATH,'//*[@id="address"]')
telephone = browser.find_element(By.XPATH,'//*[@id="phone"]')
adresse_facture = browser.find_element(By.XPATH,'//*[@id="addressFacturation"]')
adresse_livraison = browser.find_element(By.XPATH,'//*[@id="addressLivraison"]')
update = browser.find_element(By.XPATH,'/html/body/div/div/div/div/div[2]/div/div[2]/div[1]/div/body/div/div/form/button')

with open("update_profil.json", "r") as json_file:
    profil = json.load(json_file)

for updates in profil:
    
    nom.clear()
    prenom.clear()
    adress.clear()
    telephone.clear()
    adresse_facture.clear()
    adresse_livraison.clear()

    nom_input = updates["Nom"]
    prenom_input = updates["Prenom"]
    adress_input = updates["Adresse"]
    telephone_input = updates["Telephone"]
    facturation = updates["Facture"]
    livraison = updates["Livraison"]

    nom.send_keys(nom_input)
    prenom.send_keys(prenom_input)
    adress.send_keys(adress_input)
    telephone.send_keys(telephone_input)
    adresse_facture.send_keys(facturation)
    adresse_livraison.send_keys(livraison)
    time.sleep(2)

    try:
        update.click()
        pop_up = browser.find_element(By.XPATH,'/html/body/div[2]')
        assert pop_up.is_displayed(), f"Le profil a bien été changé"
        time.sleep(1)

    except Exception as e :
        print(e)


## Étape 11 - Réinitialiser le mot de passe

In [10]:
connecting = browser.find_element(By.ID,'style_avatar_wrapper__pEGIQ')
connecting.click()

connect = browser.find_element(By.ID,'mui-p-43609-T-1')
connect.click()

forgor = browser.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div/div[2]/div[1]/div/main/div[2]/div[3]/div/div/form/a')
forgor.click()

forgotten_email_input = browser.find_element(By.XPATH,'//*[@id="email_reset_pass"]')
new_pswd_input = browser.find_element(By.XPATH,'//*[@id="reset_password"]')
reset = browser.find_element(By.XPATH,'//*[@id="btn_reset_password"]')

with open("users.json", "r") as json_file:
    loaded_users = json.load(json_file)

account = loaded_users[2]

email_input = account["email"]
new_pswd = "4231abcd"

try : 
    forgotten_email_input.clear()
    new_pswd_input.clear()

    forgotten_email_input.send_keys(email_input)
    new_pswd_input.send_keys(new_pswd)
    reset.click()
    
    time.sleep(3)

    nouvel_url = browser.current_url

    page_attendue_url = "https://ztrain-web.vercel.app/auth/login"
    assert nouvel_url == page_attendue_url, f"L'URL de la nouvelle page est {nouvel_url}, mais on s'attend à {page_attendue_url}"

except Exception as e:
    print(e)


IndexError: list index out of range